In [7]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import statsmodels.api as sm

In [8]:
# This dataset you have seen, but this time we will properly split the data from the actual
# dataset into two, and fit the model on the train dataset, and test on the test dataset.
# Then we will iterate through class thresholds, to see which threshold gives the best confusion
# matrix. The first steps have been done for you 
# (creating dummies, joining to df, creating y series and features only dataframe 
# but please be familiar with these first steps! 

df_raw = pd.read_csv("../../assets/admissions.csv")
df = df_raw.dropna() 

dummies = pd.get_dummies( df["prestige"], prefix = "prestige" )

print df.head()

join = df[ df.columns[0:3] ].join(dummies)
join["intercept"] = 1

joinColumns = join.columns
y = join.admit
X = join[ joinColumns[1:3] + joinColumns[4:] ]

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


/Users/SangYeob/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: FutureWarning: using '+' to provide set union with Indexes is deprecated, use '|' or .union()


In [26]:
# split data randomly into datasets, 70% train, 30% test using test train split
# HINT: X_train, X_test, y_train, y_test = train_test_split( parameters )
# call them X_train, X_test, y_train, and y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state = 30)

In [31]:
# fit the model using statsmodels.api.sm
# HINT: sm.logit(y_train, X_train) then fit it
logReg = sm.Logit(y_train,X_train)
result = logReg.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.567931
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  277
Model:                          Logit   Df Residuals:                      271
Method:                           MLE   Df Model:                            5
Date:                Thu, 30 Jun 2016   Pseudo R-squ.:                 0.09148
Time:                        11:39:28   Log-Likelihood:                -157.32
converged:                       True   LL-Null:                       -173.16
                                        LLR p-value:                 6.873e-06
================================================================================
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gpa              0.9387      0.401      2.342      0.019         0.153     1.724
gre              0.0024      0.001      1.805      0.071        -0.000     0.005
intercept       -4.6529      1.349     -3.450      0.001        -7.296    -2.010
prestige_2.0    -0.5204      0.379     -1.374      0.169        -1.263     0.222
prestige_3.0    -1.2389      0.414     -2.996      0.003        -2.050    -0.428
prestige_4.0    -1.4674      0.527     -2.782      0.005        -2.501    -0.434
================================================================================
"""

In [38]:
import seaborn as sns
%matplotlib

#sns.lmplot('gre','admit',join,logistic=True)
sns.heatmap(join.corr())

Using matplotlib backend: MacOSX


In [28]:
# add y_test as a new column in X_test, and then make another dataframe called dfTrain
# and set it equal to X_test, after X_test has the new y_test column
# HINT: X_test["actualAdmit"] = y_test
X_test["actualAdmit"] = y_test
dfTrain = X_test
# create a new column in dfTrain that is the predicted admitance value using the result logit model
# note you will need a dataframe with only the features (including intercept)
# note the dummy column has already been removed
# HINT: 
#dfTrain['predictedAdmit'] = result.predict( dfTrain[ listOfFeatureCols ] )
dfTrain['predictedAdmit'] = result.predict(dfTrain.ix[:,:-1])


/Users/SangYeob/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/SangYeob/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [39]:
print np.exp(result.params)
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

gpa             2.556585
gre             1.002400
intercept       0.009534
prestige_2.0    0.594311
prestige_3.0    0.289693
prestige_4.0    0.230531
dtype: float64
                  2.5%     97.5%        OR
gpa           1.165402  5.608475  2.556585
gre           0.999794  1.005013  1.002400
intercept     0.000678  0.134046  0.009534
prestige_2.0  0.282924  1.248409  0.594311
prestige_3.0  0.128793  0.651608  0.289693
prestige_4.0  0.081993  0.648159  0.230531


In [29]:
# create a function call scalePredictor, that will take two parameters called "prob" and "threshold"
# the function will check if the probability is greater than or equal to the threshold, 
# return 1 if true, else return 0

# create a while loop, starting at i = 0.30 and ending i <= 0.50, in increments of 0.01. 
# In this while loop you will create a new predictionAdmit_Threshold column in every iteration
# this column will be populated by using scalePredictor each time
# after the new column is populated, print out a confusion matrix (use crosstab (within the loop!) )
# note the first parameter in crosstab will always be dfTrain['actualAdmit'] while the second
# parameter will be the new column in that iteration
# interpret each iteration, and decide on the best threshold in each iteration.

# HINTS:

#i = 0.30
#while i <= 0.50:
    
#    dfTrain[ 'predictedAdmit_{}'.format(i) ] = finish code here
    
#    print pd.crosstab(
#        dfTrain['actualAdmit'],
#        put new dfTrain column here, 
#        rownames=['admit']
#    )
#    i += 0.01

